In [ ]:
import numpy as np
from scipy.stats import norm

def likelihood(xi_prob, mu, sigma, y):
    phi = norm.pdf((y - mu) / sigma)
    y_like = np.dot(xi_prob, phi)

    return y_like

def forward_pass_algorithm(pi0, N, T, P, mu, sigma, Y):
    xi_prob_t = np.zeros((T, N))
    xi_prob_t1 = np.zeros((T, N))

    # Case t=1
    y_like = likelihood(pi0, mu, sigma, Y[0])
    for ss in range(0, N):
        phi = np.zeros((N))
        for ss2 in range(0, N):
            phi[ss2] = norm.pdf((Y[0] - mu[ss2]) / sigma)
    xi_prob_t[0, :] = np.multiply(pi0, phi) / y_like
    for ss in range(0, N):
        xi_prob_t1[0, ss] = np.dot(P[:, ss], xi_prob_t[0, :])

    for tt in range(1, T):
        y_like = likelihood(xi_prob_t1[tt - 1, :], mu, sigma, Y[tt])
        for ss in range(0, N):
            phi = np.zeros((N))
            for ss2 in range(0, N):
                phi[ss2] = norm.pdf((Y[tt] - mu[ss2]) / sigma)
        xi_prob_t[tt, :] = np.multiply(xi_prob_t1[tt - 1, :], phi) / y_like
        for ss in range(0, N):
            xi_prob_t1[tt, ss] = np.dot(P[:, ss], xi_prob_t[tt, :])

        # print(mu, phi, Y[tt-1], Y[tt], xi_prob[tt,:])

    return xi_prob_t, xi_prob_t1



def log_likelihood(theta, pi0, sigma, T, Y):
    # 0. Rearrange vector of parameters

    print(theta)

    P = np.zeros((2, 2))
    P[0, 0] = theta[0]
    P[0, 1] = 1 - P[0, 0]
    P[1, 1] = theta[1]
    P[1, 0] = 1 - P[1, 1]

    mu = [-2,2]

    # 1. Forward algorithm

    xi_prob_t, xi_prob_t1 = forward_pass_algorithm(pi0, 2, T, P, mu, sigma, Y)

    # print(xi_prob)

    # 2. Evaluate likelihood

    y_like = np.zeros(T)
    y_like[0] = likelihood(pi0, mu, sigma, Y[0])
    for tt in range(1, T):
        y_like[tt] = likelihood(xi_prob_t1[tt - 1, :], mu, sigma, Y[tt])

    sum_log_like = -np.sum(np.log(y_like))
    # print(sum_log_like)

    return sum_log_like


In [ ]:
pi0 = np.array([0.2, 0.8])
theta = [0.25,0.75]
Y = np.array([0.25,-0.3,1.5])
T = len(Y)
sigma = 1
Y = np.zeros(T)
log_likelihood(theta, pi0, sigma, T, Y)

In [ ]:
# seed random number generator
from numpy.random import seed, rand
seed(12345)

TARGET_PURSE = 15
INIT_PURSE = 5

N_STATES = TARGET_PURSE + 1

S = np.zeros((N_STATES, 1))
P = np.zeros((N_STATES, N_STATES))

P[0, 0] = 1.0
P[N_STATES - 1, N_STATES - 1] = 1.0

for ii in range(1, N_STATES - 1):
    for jj in range(0, N_STATES):
        if jj == ii - 1 or jj == ii + 1:
            P[ii, jj] = 0.5

print("Transition matrix:\n", P)
N_HISTORIES = 5000  # number of histories or simulations
INIT_PURSE = 5
TARGET_PURSE = 15
LEN_HIST = 500  # Length of each simulation
histories = np.zeros((N_HISTORIES, LEN_HIST))
histories[:, 0] = INIT_PURSE * np.ones(N_HISTORIES)
randarray = rand(N_HISTORIES, LEN_HIST)

for i in range(0, N_HISTORIES):
    for j in range(1, LEN_HIST):
        histories[i, j] = (
            histories[i, j - 1] + (randarray[i, j] >= 0.5) - (randarray[i, j] < 0.5)
        )
        if histories[i, j] == TARGET_PURSE or histories[i, j] < 1:
            histories[i, j + 1 : LEN_HIST + 1] = histories[i, j]  # noQA E203
            break

target_num = np.sum(np.max(histories, axis=1) == TARGET_PURSE)

end_gamble = np.zeros(N_HISTORIES)
end_gamble_sum = 0

for i in range(0, N_HISTORIES):
    if np.max(histories[i, :]) == TARGET_PURSE:
        where_gamble_ends_T = np.where((histories[i, :] == TARGET_PURSE))
        end_gamble[i] = where_gamble_ends_T[0][0]
        end_gamble_sum += 1
    elif np.min(histories[i, :]) < 1:
        where_gamble_ends_0 = np.where((histories[i, :] < 1))
        end_gamble[i] = where_gamble_ends_0[0][0]
        end_gamble_sum += 1
    else:
        end_gamble[i] = 0.0

broke_num = np.sum(np.min(histories, axis=1) < 1)

print(
    "Probability of getting the target:",
    target_num / N_HISTORIES,
    "\nProbability of losing all the money:",
    broke_num / N_HISTORIES,
)
print(
    "Expected time until reaching a stopping result:",
    np.sum(end_gamble) / end_gamble_sum,
    "\nTotal number of simulations:",
    end_gamble_sum,
)

In [ ]:
import numpy as np

# Define your matrix


# Calculate the eigenvalues
eigenvalues, _ = np.linalg.eig(matrix)

# Print the eigenvalues
print("Eigenvalues:", eigenvalues)

In [ ]:
P = np.array([[0.81, 0.19], [0.72, 0.28]])
# Find the eigenvalues and eigenvectors of the transpose of P
eigenvalues, eigenvectors = np.linalg.eig(P.T)

# Find the eigenvector corresponding to the eigenvalue 1
stationary_distribution = eigenvectors[:, np.where(np.isclose(eigenvalues, 1))]

# Normalize the stationary distribution so that it sums to 1
stationary_distribution = stationary_distribution / np.sum(stationary_distribution)

# The stationary distribution is the normalized eigenvector
print("Stationary Distribution:", stationary_distribution)